# STAT207 Final Project - INSERT TITLE HERE


 1. Albert Chen 
 2. Josh Fajardo 
 3. Shray Srivastava 
 4. Justin Starks 

**Delete this code block before submitting**

You should delete and add cells as you see fit.

Don't forget to put written text in **markdown** cells. To change a cell from a <u>code block</u> to a <u>markdown cell</u>:
* Click on the cell you'd like to modify.
* Go to 'Cell' at the top of the page.
* Go to 'Cell Type' in the drop down menu and click on either 'Code' or 'Markdown'.


In [121]:
#Imports here
import pandas as pd 
import numpy as np 

## 1. Introduction

Research Introduction & Motivation

This model can be built to appeal to various audiences. The motivation of creating this model can aid sports analysts in their predictions of NBA All-Stars in the upcoming season, sport betters who want to bet on potential all stars for the upcoming NBA season, and even NBA players to see how well they need to play to become an All-Star. In the context of predicting NBA All-Stars, the "positives" would be the players who are selected as All-Stars, and the "negatives" would be those who are not selected. Since the goal of our model is to predict NBA All-Stars, we believe that those would utilize this model would prefer a model that would be better at classifying positive, or All-Stars, in this case. According to Sara Brown of MIT, "programmers choose a machine learning model to use, supply the data, and let the computer model train itself to find patterns or make predictions" (Machine Learning, explained, 2021). If this is the case would we prefer our model to be better at predicting the wrong outcome? Furthermore a potential user of this model can be a sports better, according to Neil Sahota, a professor at UC Irvine, "AI, with its unparalleled ability to analyze vast datasets and discern patterns beyond human capability, is emerging as the new MVP in this field" (The Game Changer: How AI Is Transforming The World Of Sports Gambling, 2024). 

Research Goal Statement

## 2. Dataset Discussion

In [122]:
per_game_stats = pd.read_csv('NBA_PG_Stats.csv')
per_game_stats.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,PF-C,24,TOT,74,18,21.9,3.2,6.3,...,0.616,2.6,4.0,6.6,1.3,0.6,0.9,1.1,1.9,7.6
1,1,Precious Achiuwa,C,24,TOR,25,0,17.5,3.1,6.8,...,0.571,2.0,3.4,5.4,1.8,0.6,0.5,1.2,1.6,7.7
2,1,Precious Achiuwa,PF,24,NYK,49,18,24.2,3.2,6.1,...,0.643,2.9,4.3,7.2,1.1,0.6,1.1,1.1,2.1,7.6
3,2,Bam Adebayo,C,26,MIA,71,71,34.0,7.5,14.3,...,0.755,2.2,8.1,10.4,3.9,1.1,0.9,2.3,2.2,19.3
4,3,Ochai Agbaji,SG,23,TOT,78,28,21.0,2.3,5.6,...,0.661,0.9,1.8,2.8,1.1,0.6,0.6,0.8,1.5,5.8


In [123]:
per_game_stats.shape[0]

500

In [124]:
advanced_stats = pd.read_csv('NBA_PG_ADV.csv')
advanced_stats.head()

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP
0,1,Precious Achiuwa,PF-C,24,TOT,74,1624,14.6,0.545,0.207,...,NaN,1.2,2.2,3.4,0.102,NaN,-1.7,0.3,-1.4,0.2
1,1,Precious Achiuwa,C,24,TOR,25,437,15.0,0.512,0.276,...,NaN,0.0,0.4,0.4,0.048,NaN,-1.4,-0.2,-1.6,0.0
2,1,Precious Achiuwa,PF,24,NYK,49,1187,14.5,0.564,0.167,...,NaN,1.2,1.8,3.0,0.122,NaN,-1.9,0.5,-1.4,0.2
3,2,Bam Adebayo,C,26,MIA,71,2416,19.8,0.576,0.041,...,NaN,2.9,4.3,7.2,0.144,NaN,0.8,1.7,2.4,2.7
4,3,Ochai Agbaji,SG,23,TOT,78,1641,7.7,0.497,0.487,...,NaN,-0.5,0.6,0.1,0.002,NaN,-3.5,-0.9,-4.4,-1.0


In [125]:
advanced_stats.shape[0]

500

We found these csv files from Sports-Reference.com. This file was download on 4/17/2024, and the player per game stats were found on https://www.basketball-reference.com/leagues/NBA_2024_per_game.html and the advanced statistics were found on https://www.basketball-reference.com/leagues/NBA_2024_advanced.html. In these specific datasets, rows represent NBA players, the NBA uses cameras and Second Spectrum software tracks the movements of every player on the court and the basketball 25 times per second and records player statistics. The dataset could potentially not account all observations. Some players are referred as two-way players, and spend some time in the NBA or G-League. These players might not be included in the dataset. However, this will not likely have a big effect on the dataset because it's unlikely that a two-way player is going to be an All-Star. Our response variable is going to be All-Stars. These players are selected by fans for their fantastic performance throughout the season. The explanatory variables we intend to use are points, rebounds, assist, steals+blocks, Win Shares, and usage. Points are the total number of points a player scores in a game. This is one of the most direct measures of a player's offensive contributions on the court. Points can be scored through field goals, three-point shots, and free throws. Rebounds are recorded whenever a player retrieves the ball after a missed shot that hits the rim or backboard. Rebounds are classified into two categories: Offenseive and Defensive rebounds. An assist is credited to a player who passes the ball to a teammate in a way that leads to a score. This statistic is a key indicator of a player’s ability to distribute the ball and set up teammates for scoring opportunities. A steal occurs when a defensive player legally takes the ball away from an offensive player, which leads to a change in possession. A block is recorded when a defensive player legally deflects an offensive player's shot attempt, preventing the ball from going into the basket. Adding steals and blocks together gives a combined measure of a player’s defensive plays that directly interrupt the opponent’s scoring attempts. Win Shares is an advanced statistic that estimates the number of wins contributed by a player to their team’s total success. It combines various aspects of play (offense and defense) into a single number that quantifies a player's overall impact on winning games. It is calculated using player, team, and league statistics in both offensive and defensive dimensions.Usage rate is an estimate of the percentage of team plays used by a player while they were on the floor. It is a measure of how often a player attempts to score, assists on a basket, or commits a turnover. High usage rates can indicate that a player has a significant role in their team's offense. We chose to focus on these variables because we believe they will help accurately predict All-Stars.

## 3. Dataset Cleaning

Missing Values Detection and Cleaning for Per Game Statitics and Advanced Statistics

For both of our datasets, the inevitable has happened to certain players: they were traded. As a result there are duplicates present in these datasets. However, Basketball-Reference does a fantastic job of mitigating this by adding a 'Tot' observation. This 'Tot' or Total means the total statistics from this player and all teams they have played on throughout the season. 

In [126]:
per_game_stats = per_game_stats[[ 'Player' , 'Tm' , 'PTS' , 'TRB' , 'AST' , 'STL' , 'BLK' ]]
per_game_stats

,Player,Tm,PTS,TRB,AST,STL,BLK
0,Precious Achiuwa,TOT,7.6,6.6,1.3,0.6,0.9
1,Precious Achiuwa,TOR,7.7,5.4,1.8,0.6,0.5
2,Precious Achiuwa,NYK,7.6,7.2,1.1,0.6,1.1
3,Bam Adebayo,MIA,19.3,10.4,3.9,1.1,0.9
4,Ochai Agbaji,TOT,5.8,2.8,1.1,0.6,0.6
...,...,...,...,...,...,...,...
495,Jordan Nwora,TOR,7.9,3.4,1.3,0.6,0.4
496,Royce O'Neale,TOT,7.7,4.8,2.8,0.7,0.6
497,Royce O'Neale,BRK,7.4,4.5,2.8,0.7,0.6
498,Royce O'Neale,PHO,8.1,5.2,2.7,0.9,0.5


In [127]:
per_game_stats = per_game_stats.dropna()
per_game_stats

,Player,Tm,PTS,TRB,AST,STL,BLK
0,Precious Achiuwa,TOT,7.6,6.6,1.3,0.6,0.9
1,Precious Achiuwa,TOR,7.7,5.4,1.8,0.6,0.5
2,Precious Achiuwa,NYK,7.6,7.2,1.1,0.6,1.1
3,Bam Adebayo,MIA,19.3,10.4,3.9,1.1,0.9
4,Ochai Agbaji,TOT,5.8,2.8,1.1,0.6,0.6
...,...,...,...,...,...,...,...
495,Jordan Nwora,TOR,7.9,3.4,1.3,0.6,0.4
496,Royce O'Neale,TOT,7.7,4.8,2.8,0.7,0.6
497,Royce O'Neale,BRK,7.4,4.5,2.8,0.7,0.6
498,Royce O'Neale,PHO,8.1,5.2,2.7,0.9,0.5


In [128]:
advanced_stats = advanced_stats[['Player', 'Tm' , 'WS' , 'USG%']]
advanced_stats

,Player,Tm,WS,USG%
0,Precious Achiuwa,TOT,3.4,15.9
1,Precious Achiuwa,TOR,0.4,21.2
2,Precious Achiuwa,NYK,3.0,14.0
3,Bam Adebayo,MIA,7.2,24.9
4,Ochai Agbaji,TOT,0.1,13.4
...,...,...,...,...
495,Jordan Nwora,TOR,0.4,22.3
496,Royce O'Neale,TOT,3.4,13.9
497,Royce O'Neale,BRK,1.8,13.9
498,Royce O'Neale,PHO,1.7,13.9


In [129]:
advanced_stats = advanced_stats.dropna()
advanced_stats

,Player,Tm,WS,USG%
0,Precious Achiuwa,TOT,3.4,15.9
1,Precious Achiuwa,TOR,0.4,21.2
2,Precious Achiuwa,NYK,3.0,14.0
3,Bam Adebayo,MIA,7.2,24.9
4,Ochai Agbaji,TOT,0.1,13.4
...,...,...,...,...
495,Jordan Nwora,TOR,0.4,22.3
496,Royce O'Neale,TOT,3.4,13.9
497,Royce O'Neale,BRK,1.8,13.9
498,Royce O'Neale,PHO,1.7,13.9


For both of our datasets, the inevitable has happened to certain players: they were traded. As a result there are duplicates present in these datasets. However, Basketball-Reference does a fantastic job of mitigating this by adding a 'Tot' observation. This 'Tot' or Total means the total statistics from this player and all teams they have played on throughout the season. 

In [130]:
per_game_stats.drop_duplicates(subset='Player', keep='first', inplace=True)

print(per_game_stats)

                       Player   Tm   PTS   TRB  AST  STL  BLK
0            Precious Achiuwa  TOT   7.6   6.6  1.3  0.6  0.9
3                 Bam Adebayo  MIA  19.3  10.4  3.9  1.1  0.9
4                Ochai Agbaji  TOT   5.8   2.8  1.1  0.6  0.6
7                Santi Aldama  MEM  10.7   5.8  2.3  0.7  0.9
8    Nickeil Alexander-Walker  MIN   8.0   2.0  2.5  0.8  0.5
..                        ...  ...   ...   ...  ...  ...  ...
491           Frank Ntilikina  CHO   1.0   1.2  0.8  0.0  0.0
492              Jusuf Nurki_  PHO  10.9  11.0  4.0  1.1  1.1
493              Jordan Nwora  TOT   7.0   2.9  1.2  0.5  0.3
496             Royce O'Neale  TOT   7.7   4.8  2.8  0.7  0.6
499               Chuma Okeke  ORL   2.3   1.7  0.4  0.2  0.1

[382 rows x 7 columns]


In [133]:
advanced_stats.drop_duplicates(subset = 'Player' , keep = 'first' , inplace = True)
print(advanced_stats)

                       Player   Tm   WS  USG%
0            Precious Achiuwa  TOT  3.4  15.9
3                 Bam Adebayo  MIA  7.2  24.9
4                Ochai Agbaji  TOT  0.1  13.4
7                Santi Aldama  MEM  2.4  17.9
8    Nickeil Alexander-Walker  MIN  4.3  14.8
..                        ...  ...  ...   ...
491           Frank Ntilikina  CHO -0.1  11.2
492              Jusuf Nurki_  PHO  5.8  19.1
493              Jordan Nwora  TOT  0.5  22.4
496             Royce O'Neale  TOT  3.4  13.9
499               Chuma Okeke  ORL  0.3  13.3

[382 rows x 4 columns]


In [134]:
#Merging advanced stats and per_game_stats
All_Star_Stats = pd.merge(per_game_stats , advanced_stats , on = 'Player' , how = 'outer')
All_Star_Stats

,Player,Tm_x,PTS,TRB,AST,STL,BLK,Tm_y,WS,USG%
0,A.J. Green,MIL,4.5,1.1,0.5,0.2,0.1,MIL,1.1,15.2
1,A.J. Lawson,DAL,3.2,1.2,0.5,0.2,0.1,DAL,0.2,20.0
2,AJ Griffin,ATL,2.4,0.9,0.3,0.1,0.1,ATL,-0.4,17.3
3,Aaron Gordon,DEN,13.9,6.5,3.5,0.8,0.6,DEN,7.1,17.8
4,Aaron Holiday,HOU,6.6,1.6,1.8,0.5,0.1,HOU,2.5,16.7
...,...,...,...,...,...,...,...,...,...,...
377,Wesley Matthews,ATL,3.1,1.5,0.6,0.4,0.3,ATL,0.4,11.7
378,Xavier Moon,LAC,2.4,1.3,1.5,0.2,0.2,LAC,-0.1,19.1
379,Zach Collins,SAS,11.2,5.4,2.8,0.5,0.8,SAS,1.8,22.7
380,Zach LaVine,CHI,19.5,5.2,3.9,0.8,0.3,CHI,1.5,23.8


After dropping the duplicates from players being traded throughout the season, we ended up with a merged dataframe of all of our explanatory variables. However, after using the dropna command no rows were removed from both datasets. We dropped 118 observations however because they were traded and were duplicated in the datasets. By doing this, it ensures the analysis is based on complete data without any other potential roadblocks that could skew our prediction. Some pros of removing rows with missing values ensures that the analysis is based on complete data, which would later one avoid potential errors that stems from missing values. A con from dropping row with missing values is that it may reduce the sample size and potentially lead to a loss of information, especially if the missing data is not random.

## 4. Preliminary Analysis

## 5. Model Data Preprocessing

## 6. Feature Selection with k-Fold Cross-Validation

## 7. Best Model Discussion

## 8. Additional Analysis/Insight

## 9. Conclusion

## References